In [30]:
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv'
df = pd.read_csv(url)

In [31]:
df.shape

(4008, 11)

In [32]:
df.columns

Index(['Customer', 'ST', 'GENDER', 'Education', 'Customer Lifetime Value',
       'Income', 'Monthly Premium Auto', 'Number of Open Complaints',
       'Policy Type', 'Vehicle Class', 'Total Claim Amount'],
      dtype='object')

In [33]:
df.columns = (
    df.columns.str.lower()
              .str.replace(' ', '_')
              .str.replace(r'^st$', 'state', regex=True)
              
)

df.columns

Index(['customer', 'state', 'gender', 'education', 'customer_lifetime_value',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'policy_type', 'vehicle_class', 'total_claim_amount'],
      dtype='object')

In [34]:
#cleaning invalid values
df['gender'] = df['gender'].str.strip().str.upper().replace({
    'FEMALE': 'F',
    'FEMAL': 'F',
    'F': 'F',
    'MALE': 'M',
    'M': 'M'
})
print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [35]:
state_mapping = {
    'AZ': 'Arizona',
    'Cali': 'California',
    'WA': 'Washington'
}

df['state'] = df['state'].replace(state_mapping)
print(df['state'].value_counts())

state
California    331
Oregon        320
Arizona       211
Washington    111
Nevada         98
Name: count, dtype: int64


In [36]:
#updating data types to float 
df['customer_lifetime_value'] = df ['customer_lifetime_value'].str.replace('%', '', regex=False).astype(float)

In [37]:
#replace 'bachelors' with 'berchelor'
df['education'] = df['education'].replace('Bachelors', 'Bachelor')

In [38]:
#replace vihicle classes with 'Luxury'ArithmeticError
df['vehicle_class'] = df['vehicle_class'].replace({
    'Sport Car': 'Luxury',
    'Luxury SUV': 'Luxury',
    'Luxury Car': 'Luxury'
})

In [39]:
#fix customer life time value#
df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')


In [40]:
print(df['number_of_open_complaints'].unique())


['1/0/00' '1/2/00' '1/1/00' '1/3/00' '1/5/00' '1/4/00' nan]


In [41]:
#fix number
def extract_middle_number(val):
    try:
        if isinstance(val, str) and '/' in val:
            parts = val.split('/')
            if len(parts) >= 2 and parts[1].isdigit():
                return int(parts[1])
        return int(val)
    except:
        return pd.NA  # pandas-friendly missing value

df['number_of_open_complaints'] = df['number_of_open_complaints'].apply(extract_middle_number).astype('Int64')


In [42]:
df['income'] = df['income'].astype(float)
df['monthly_premium_auto'] = df['monthly_premium_auto'].astype('Int64')  # allows NaN
df['total_claim_amount'] = df['total_claim_amount'].astype(float)

# Verify types
df.dtypes


customer                      object
state                         object
gender                        object
education                     object
customer_lifetime_value      float64
income                       float64
monthly_premium_auto           Int64
number_of_open_complaints      Int64
policy_type                   object
vehicle_class                 object
total_claim_amount           float64
dtype: object

In [43]:
df.dtypes

customer                      object
state                         object
gender                        object
education                     object
customer_lifetime_value      float64
income                       float64
monthly_premium_auto           Int64
number_of_open_complaints      Int64
policy_type                   object
vehicle_class                 object
total_claim_amount           float64
dtype: object

In [44]:
#detecting null values
null_counts = df.isnull().sum()
print(null_counts[null_counts > 0])

customer                     2937
state                        2937
gender                       3054
education                    2937
customer_lifetime_value      2940
income                       2937
monthly_premium_auto         2937
number_of_open_complaints    2937
policy_type                  2937
vehicle_class                2937
total_claim_amount           2937
dtype: int64


In [45]:
#fill numeric nulls with median
num_cols = df.select_dtypes(include=['float64', 'Int64', 'int']).columns
for col in num_cols:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

#fills categorical columns with mode
cat_cols = df.select_dtypes(include=['object', 'category']).columns
for col in cat_cols:
    mode_val = df[col].mode()[0]
    df[col] = df[col].fillna(mode_val)

In [46]:
print(df.isnull().sum().sum())  # Should be 0 if all nulls are handled


0


In [47]:
#identify duplicates rows
duplicates = df.duplicated()
print(f"Number of duplicates rows: {duplicates.sum()}")

Number of duplicates rows: 2936


In [48]:
#drop duplicate rows, keep the first 
df = df.drop_duplicates(keep='first')
#reset index
df = df.reset_index(drop=True)

In [49]:
print(f"Remaining duplicate rows: {df.duplicated().sum()}")

Remaining duplicate rows: 0


In [50]:
#save the cleaned dataset 
df.to_csv('cleaned_dataset.csv', index=False)

In [51]:
# Step 1: Review statistics for reference
print("Total Claim Amount stats:")
print(df['total_claim_amount'].describe())
print("\nCustomer Lifetime Value stats:")
print(df['customer_lifetime_value'].describe())

# Step 2: Calculate thresholds
claim_threshold = df['total_claim_amount'].quantile(0.75)
clv_threshold = df['customer_lifetime_value'].quantile(0.25)

# Step 3: Filter the DataFrame
target_customers = df[
    (df['total_claim_amount'] > claim_threshold) &
    (df['customer_lifetime_value'] < clv_threshold)
]

# Step 4: Describe both columns
summary_stats = target_customers[['total_claim_amount', 'customer_lifetime_value']].describe()

# Step 5: Output
print(f"\nNumber of target customers: {target_customers.shape[0]}")
print("\nSummary statistics for target customers:")
print(summary_stats)


Total Claim Amount stats:
count    1072.000000
mean      404.940027
std       292.894449
min         0.382107
25%       202.509051
50%       354.729129
75%       532.800000
max      2893.239678
Name: total_claim_amount, dtype: float64

Customer Lifetime Value stats:
count    1.072000e+03
mean     7.929234e+05
std      6.423980e+05
min      2.004351e+05
25%      4.054051e+05
50%      5.881742e+05
75%      8.937675e+05
max      5.816655e+06
Name: customer_lifetime_value, dtype: float64

Number of target customers: 40

Summary statistics for target customers:
       total_claim_amount  customer_lifetime_value
count           40.000000                40.000000
mean           723.776577            341577.910000
std            149.280606             56921.472027
min            537.600000            228759.690000
25%            633.378846            304220.875000
50%            685.284189            365995.370000
75%            799.200000            387364.700000
max           1185.988301    